In [5]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [20]:
input_tensor = layers.Input(shape=(150, 300, 3))
# input_tensor = layers.Conv2D(32, (3, 3), padding='same')(input_tensor)
# input_tensor = layers.MaxPooling2D((1, 2))(input_tensor)

model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [21]:
base_model = model
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 150, 300, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 149, 32)  864         input_12[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 149, 32)  128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [4]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x =  layers.Dropout(0.4)(x)

predictions = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 300, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 149, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 149, 32)  128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 149, 32)  0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'vazios_cropped',
    target_size=(150, 300),
    batch_size=8,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'vazios_cropped',
    target_size=(150, 300),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)


Found 756 images belonging to 2 classes.
Found 188 images belonging to 2 classes.


In [20]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                              verbose=1, min_delta=1e-2, mode='min')

model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])

model.fit(train_generator, epochs=50,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 60)

Train for 6 steps, validate for 3 steps
Epoch 1/50
6/6 [==============================] - 19s 3s/step - loss: 0.5891 - acc: 0.7804 - val_loss: 0.4336 - val_acc: 0.9000
Epoch 2/50
6/6 [==============================] - 15s 3s/step - loss: 0.4408 - acc: 0.9206 - val_loss: 0.3547 - val_acc: 0.8556
Epoch 3/50
6/6 [==============================] - 16s 3s/step - loss: 0.3612 - acc: 0.9497 - val_loss: 0.2677 - val_acc: 0.9222
Epoch 4/50
6/6 [==============================] - 16s 3s/step - loss: 0.2827 - acc: 0.9431 - val_loss: 0.2551 - val_acc: 0.9111
Epoch 5/50
6/6 [==============================] - 16s 3s/step - loss: 0.2427 - acc: 0.9709 - val_loss: 0.2399 - val_acc: 0.9111
Epoch 6/50
6/6 [==============================] - 16s 3s/step - loss: 0.2009 - acc: 0.9749 - val_loss: 0.2482 - val_acc: 0.8778
Epoch 7/50
6/6 [==============================] - 15s 3s/step - loss: 0.1945 - acc: 0.9788 - val_loss: 0.2296 - val_acc: 0.9056
Epoch 8/50
6/6 [==============================] - 15s 3s/step - 

In [21]:
for ind, layer in enumerate(model.layers):
    print(ind, layer.name, layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_4 False
13 block2_pool False
14 batch_normalization_4 False
15 add_12 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_5 False
23 block3_pool False
24 batch_normalization_5 False
25 add_13 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_6 False
33 block4_pool False
34 batch_normalization_6 False
35 add_14 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepco

In [7]:
#Fine tunning
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[126:]:
    layer.trainable = True

In [8]:
for ind, layer in enumerate(model.layers):
    print(ind, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d False
13 block2_pool False
14 batch_normalization False
15 add False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_1 False
23 block3_pool False
24 batch_normalization_1 False
25 add_1 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_2 False
33 block4_pool False
34 batch_normalization_2 False
35 add_2 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2 False

In [9]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4,
                          verbose=1, min_delta=1e-2, mode='min')

model.compile(optimizer=optimizers.Adam(lr=0.000001), loss='binary_crossentropy', metrics=['acc'])

In [10]:
# model.save_weights('inceptionvazios_cropped.hdf5')
model.load_weights('inceptionvazios_cropped.hdf5')

In [11]:
model.fit_generator(train_generator, epochs=20,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 60)

Epoch 1/20
95/95 [==============================] - 30s 320ms/step - loss: 0.1998 - acc: 0.9114 - val_loss: 0.2109 - val_acc: 0.8667
Epoch 2/20
95/95 [==============================] - 29s 301ms/step - loss: 0.1569 - acc: 0.9458 - val_loss: 0.2650 - val_acc: 0.8667
Epoch 3/20
95/95 [==============================] - 28s 299ms/step - loss: 0.1240 - acc: 0.9511 - val_loss: 0.2632 - val_acc: 0.9333
Epoch 4/20
95/95 [==============================] - 28s 300ms/step - loss: 0.2095 - acc: 0.9272 - val_loss: 0.1487 - val_acc: 0.9333
Epoch 5/20
95/95 [==============================] - 29s 303ms/step - loss: 0.1462 - acc: 0.9524 - val_loss: 0.1533 - val_acc: 0.9333
Epoch 6/20
95/95 [==============================] - 29s 301ms/step - loss: 0.1387 - acc: 0.9497 - val_loss: 0.1512 - val_acc: 0.9333
Epoch 7/20
95/95 [==============================] - 29s 304ms/step - loss: 0.1471 - acc: 0.9418 - val_loss: 0.2220 - val_acc: 0.8667
Epoch 8/20
94/95 [============================>.] - ETA: 0s - loss: 0